## Accessing Sentinel-5P data on Azure

The [Sentinel-5P](https://sentinel.esa.int/web/sentinel/missions/sentinel-5p) mission provides daily global atmospheric measurements at a resolution of 3.5km x 7km on most bands.  This dataset represents the global archive of Sentinel-5P [Level 2](http://www.tropomi.eu/data-products/level-2-products) products, from 2018 to the present, in NetCDF format.  

This notebook demonstrates basic access to Sentinel-5 data on Azure.  Because Sentinel-1 data are in preview, the user needs to provide storage credentials.

This dataset is stored in the West Europe Azure region, so this notebook will run most efficiently on Azure compute located in the same region.  If you are using this data for environmental science applications, consider applying for an [AI for Earth grant](http://aka.ms/ai4egrants) to support your compute requirements.

This dataset is documented at [aka.ms/ai4edata-sentinel-5p](http://aka.ms/ai4edata-sentinel-5p).

Sentinel-5P data on Azure are maintained by [Sinergise](https://sinergise.com/).

### Environment setup

In [ ]:
import os
import fsspec
import xarray as xr
import numpy as np
from azure.storage.blob import ContainerClient

# Not used directly, but needs to be installed to read NetCDF files with xarray
import h5netcdf

### Auth files

In [ ]:
# A plain-text file with a SAS token (starting with "?sv") on the first line
sas_file = os.path.expanduser('~/tokens/sentinel-5p_sas.txt')

### Constants

In [ ]:
# Let's look at ozone concentration from mid-day on Jan 1, 2021
product = 'L2__O3____'
date = '2021/01/01'

### Azure storage constants

In [ ]:
lines = []
with open(sas_file,'r') as f:
    lines = f.readlines()
assert len(lines) >= 1
sas_token = lines[0].strip()
        
storage_account_name = 'sentinel5euwest'
container_name = 'sentinel-5p'
storage_account_url = 'https://' + storage_account_name + '.blob.core.windows.net/'

container_client = ContainerClient(account_url=storage_account_url, 
                                                 container_name=container_name,
                                                 credential=sas_token)

### List products matching our product/date

In [ ]:
prefix = '/'.join(['TROPOMI',product,date])
print('Searching for prefix {}'.format(prefix))
generator = container_client.list_blobs(name_starts_with=prefix)
scene_paths = [blob.name for blob in generator]
print('\nFound {} matching scenes:\n'.format(len(scene_paths)))
for s in scene_paths:
    print(s.split('/')[-1])

### Print metadata for one scene

Choose an OFFL scene for this product/date (NRTI products have smaller domains).  In practice the scene we choose impacts
the longitude we're plotting, since S5P has a daily orbit.  We'll choose a mid-day scene.

In [ ]:
offl_scenes = [s for s in scene_paths if 'OFFL' in s]
scene_path = offl_scenes[len(offl_scenes) // 2]
url = storage_account_url + container_name + '/' + scene_path
print('Processing image at URL:\n{}'.format(url))

In [ ]:
import warnings; warnings.filterwarnings('ignore')
with fsspec.open(url+sas_token) as f:
    ds = xr.open_dataset(f)
print(ds)

### Open the data

...which lives in the 'PRODUCT' NetCDF group.

In [ ]:
with fsspec.open(url+sas_token) as f:
    ds = xr.open_dataset(f,group='/PRODUCT')
print(ds)

### Plot the data in its native coordinates

In [ ]:
if 'CH4' in product:
    varname = 'methane_mixing_ratio'
elif 'NO2' in product:
    varname = 'nitrogendioxide_tropospheric_column'
elif 'O3' in product:
    varname = 'ozone_total_vertical_column'
elif 'CO' in product:
    varname = 'carbonmonoxide_total_column'
elif 'SO2' in product:
    varname = 'sulfurdioxide_total_vertical_column'
    
data = ds[varname][0,:,:]
data.plot();

### Extract values as numpy arrays

In [ ]:
z = data.values
lon = ds['longitude'].values.squeeze()
lat = ds['latitude'].values.squeeze()

# Don't plot extreme latitudes; they make it hard to zoom in a nice way
minlat = -60; maxlat = 60

# Zoom to a sensible longitude area by plotting only values that are non-nan
# and above a threshold.
plot_threshold = np.nanpercentile(z,50)

valid_indices = np.argwhere((~np.isnan(z)) & (z > plot_threshold))

minlon = None; maxlon = None
for xy in valid_indices:
    xy_lon = lon[xy[0],xy[1]]
    xy_lat = lat[xy[0],xy[1]]
    if xy_lat > maxlat or xy_lat < minlat:
        continue
    if minlon is None or xy_lon < minlon:
        minlon = xy_lon
    if maxlon is None or xy_lon > maxlon:
        maxlon = xy_lon

plot_extent = (minlon, maxlon, minlat, maxlat)

### Plot on a basemap

In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from cartopy.feature import NaturalEarthFeature

figure, ax = plt.subplots(figsize=(15, 10))
ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=0.0))

# Prepare the background and axes
boundaries = cfeature.NaturalEarthFeature(
    category='cultural',name='admin_0_countries',scale='50m',facecolor='none')
ax.add_feature(boundaries, edgecolor='lightgray')
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                  linewidth=1, color='gray', alpha=0.5, linestyle=':')
gl.xformatter = LONGITUDE_FORMATTER; gl.yformatter = LATITUDE_FORMATTER
ax.set_extent(plot_extent,ccrs.PlateCarree())

# Plot
plt.contourf(lon, lat, z, 50, transform=ccrs.PlateCarree())
plt.colorbar(fraction=0.015, pad=0.08)
plt.show()